## Nonlinear DEGs from PerturbSeq

#### Background
- We have got a reactome similarity matrix between the perturbed genes, both in desiase and healthly.
- We know that DEGs-based similarity matrix has a statistically significant overlap with the reactome similarty matrix. 
- We want to extract knowledge from the perturbSeq dataset which would overlap with the reactome pathways.
- We already tried to build a VAE-based model learning an interpretable representation of every perturbation. Even though it gives promising results (statistically significant overlap with reactome similarity mtx), it requires further tuning/refinement as the stability of the solution is low. Specifically, adjustment of the loss function and the learning concept might be needed.
- At this point we operate on single-gene perturbations only due to their broader availability. We might use 2-gene perturbations for validation of our hypotheses and expansion of the method in the following iterations.

#### The Why
- Why do we believe it is reasonable to do? DEGs are computed using t-test and have a statistically significant overlap with reactome. If we incorporate nonlinearity into DEGs in some way, we might get an even stronger result.
- Why do we want to do it? It would give us a subset of perspective relationships between genes whose exploration would potentially lead to new biological findings.
- **Optional addon:** We know that statistical methods used for DEGs do not use feature interactions (they might but it is uncommon). We could introduce a model which would focus on feature interactions only. This way we could have effect from the single-feature effects and interactions separately. 

#### The How
- What is the simplest way? Take a binary boosting trees classifier (lgbm or catboost), and find the top useful features for every perturbation, for the model classifying into ctrl vs perturbed state. Ofc, one can play with SHAPs, etc. 
- What is a more advanced approach? Having a similarity matrix from the previous step, we can refine it by building binary classfiers for perturbation A vs perturbation B task. This would allow to refine differences between the perturbations. The classifier performance and its feature importances (or SHAPs) would be used for the refinement.
- Check the result consistency with HPT and swapping the classifier.

#### Performance of the approach
- How to assess the success of the approach? Hmm...
  - Check the overlap with the DEGs - for exploration purposes only, does not allow to assess usability
  - Consider TF pairs which were not explored together and are similar according to our approach. Play with them in the wetlab to get a proof that they are connected? Is it of any use at all?
  
#### Thoughts and feedback
- Network analysis: Construct gene interaction networks based on your results and compare their properties (e.g., modularity, centrality) with known biological networks.
- Consider using multi-class classification instead of multiple binary classifiers to potentially capture global patterns.
- Implement a hierarchical classification scheme to group similar perturbations - this is basically a sequence of splits by multiple models following the known hierarchy. We have got the hierarchy from the VAE-based model or we can get one from the hierarchical clustering. Now, we can refine the paths. Here are several approaches we can consider for that:
    - Start with your initial hierarchy.
    - Train classifiers at each node and evaluate their performance.
    - Identify nodes where performance is poor.
    - For these problematic nodes, consider:
        - a) Splitting the node further if it's too heterogeneous.
        - b) Merging with a sibling node if they're too similar.
        - c) Adjusting the perturbations within the node - check the ones contributing to the error the most and change their class.
    - Repeat this process iteratively.
- **Reasons for only around 10% overlap between DEGs and the pathways?**
  - a) The genes are regulated by the same TF/protein/signalling mechanism but have nothing else in common (called indirect relationships).
    - This can still be reported in pathways DBs, e.g. WikiPathways, right? 
  - b) Incomplete pathway annotations - they might be actually related but we do not know that yet.
    - This is our research gap
  - c) Spurious correlations, cell-type or condition-specific, technical artifacts, genomic proximity, temporal nature of the coexpression.
    - We account for it by analysing multiple datasets with different cell types, timelines, conditions, etc. This should address most of these issues.  
    
#### To decide on the path, we need to see what brings us to the unknown biology faster!

### Imports

In [1]:
import scanpy as scp
import pandas as pd
import numpy as np
import catboost as cb
from tqdm import tqdm
from scipy import sparse
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from collections import Counter
import lightgbm as lgb
import matplotlib.pyplot as plt
import umap
from sklearn.model_selection import StratifiedKFold

### Magics

In [99]:
GENE_PER_CELL_BINNING = True
N_BINS = 1000
N_ITER = 50
TOP_N_GENES = 5000

In [2]:
import scanpy as scp
import pandas as pd
import numpy as np
import catboost as cb
from tqdm import tqdm
from scipy import sparse
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from collections import Counter
import lightgbm as lgb
import matplotlib.pyplot as plt
import umap
from sklearn.model_selection import StratifiedKFold

import ot
from sklearn.decomposition import PCA
from IPython.display import clear_output
from sklearn.preprocessing import QuantileTransformer

### Magics

In [7]:
GENE_PER_CELL_BINNING = False
N_BINS = 1000
N_ITER = 50
TOP_N_GENES = 576

### Prepare data

In [2]:
adata = scp.read_h5ad('./data/Norman_2019/norman_umi_go/perturb_processed.h5ad')

In [3]:
TFs = pd.read_csv('little_data/TF_db.csv',index_col=0)

In [4]:
adata = adata[:, adata.var.loc[adata.var.index.isin(TFs['Ensembl ID'].values)].index]

In [5]:
## Following the scGPT paper, we bin the genes within cell. 

def bin_nonzero_values(arr, num_bins):
    # Filter out non-zero values
    nonzero_vals = arr[arr != 0]
    
    # Calculate bin edges
    bin_edges = np.linspace(nonzero_vals.min(), nonzero_vals.max(), num_bins)
    
    # Bin the values
    binned_values = np.zeros_like(arr)
    binned_nonzero = np.digitize(nonzero_vals, bin_edges)
    binned_values[arr != 0] = binned_nonzero
    
    return binned_values

# Example usage
arr = np.random.randint(low=0, high=100, size=100)
num_bins = 3
binned_values = bin_nonzero_values(arr, num_bins)
print(set(binned_values))

{1, 2, 3}


In [8]:
scp.pp.normalize_total(adata, exclude_highly_expressed=True)
scp.pp.log1p(adata)
scp.pp.highly_variable_genes(adata, n_top_genes=TOP_N_GENES,subset=False)

In [9]:
if GENE_PER_CELL_BINNING:
    tempy = adata.X.toarray()
    
    for c in tqdm(range(adata.X.shape[0])):
        tempy[c,:] = bin_nonzero_values(tempy[c,:], N_BINS)
    
    adata.X = sparse.csr_matrix(tempy)
    del tempy

In [10]:
y = adata.obs.condition.values.astype(str)
X = adata.X.toarray()

In [12]:
gene_num_map = ['ctrl']

for rec in tqdm(y):
    comps = rec.split('+')
    for c in comps:
        if c not in gene_num_map:
            gene_num_map.append(c)

100%|██████████| 91205/91205 [00:00<00:00, 675115.72it/s]


In [124]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

# Assuming you have X (features) and y (labels) already defined
# Replace with your actual data

# Create a dictionary to store top N important features for each dataset
top_features_dict = {}
feats = adata.var.gene_name.values
# Define the number of top features to select
N = 50

# Iterate over each gene in gene_num_map
for g in tqdm(gene_num_map[1:]):
    # Create a mask for the current gene
    mask = np.array([(g in element and 'ctrl' in element) or element == 'ctrl' for element in y]).astype(bool)
    
    # Filter the data for the current gene
    X_temp = X[mask]
    y_temp = y[mask]
    y_temp[y_temp != 'ctrl'] = 1
    y_temp[y_temp == 'ctrl'] = 0
    
    # Split the data into train, validation, and test sets
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42)
    #X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)
    
    # Initialize the CatBoost classifier
    clf = CatBoostClassifier(loss_function='Logloss') #iterations=300, depth=7, learning_rate=0.1, 
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=0)
    
    # Get feature importances
    feature_importances = clf.feature_importances_
    
    # Get indices of top N important features
    top_feature_indices = np.argsort(feature_importances)[-N:]
    
    # Get the actual feature names
    top_features = feats[top_feature_indices]
    
    # Store the top features in the dictionary
    top_features_dict[g] = top_features

# Create a pandas DataFrame from the dictionary
top_features_df = pd.DataFrame.from_dict(top_features_dict).T

100%|██████████| 105/105 [06:50<00:00,  3.91s/it]


In [125]:
top_features_df.to_csv('./little_data/CatBoost_top50_important_for_binary_classify.csv')

In [126]:
sim_mtx = pd.DataFrame(index = top_features_df.index, columns = top_features_df.index)
for g1 in tqdm(top_features_df.index):
    for g2 in top_features_df.index:
        
        t1 = top_features_df.loc[g1].values.tolist()
        t2 = top_features_df.loc[g2].values.tolist()

        m = top_features_df.loc[g1].isin(t2)
        d1 = m.loc[m==True].index.values

        m = top_features_df.loc[g2].isin(t1)
        d2 = m.loc[m==True].index.values
        
        sim_mtx.loc[g1,g2] = float(len(set(t1).intersection(set(t2))))


100%|██████████| 105/105 [00:27<00:00,  3.76it/s]


In [130]:
sim_mtx.to_csv('./little_data/CatBoost_sim_mtx.csv')